In [1]:
import math
def euclidean_distance(city1, city2):
    x1= float(city1[0])
    y1 = float(city1[1])
    x2 = float(city2[0])
    y2 = float(city2[1])
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

In [2]:
import csv
from heapq import heappush
cities = []
with open(r'/content/2024_AI_TSP.csv', mode='r', newline='') as tsp:
  reader = csv.reader(tsp)
  for row in reader:
    cities.append(row)


import networkx as nx
n = len(cities)
G = nx.complete_graph(n)
for i in range(n):
  for j in range(n):
    if i != j:
      city1, city2 = cities[i], cities[j]
      distance = euclidean_distance(city1, city2)
      G.add_edge(i,j,weight = distance)

n = len(cities)
sortedNodeList = [ [] for i in range(n)]
for node in G.nodes:
  for neighbor in G.neighbors(node):
    sortedNodeList[node].append((G.edges[node, neighbor]['weight'], neighbor))

for i, node in enumerate(sortedNodeList):
    sortedNodeList[i] = sorted(node)

In [3]:
def calculate_distance(graph, path):# 경로들이 도시개수만큼 나왔을 때 전체 거리 구하는 함수
    total_distance = 0.0
    for i in range(len(path) - 1):
        total_distance += graph.edges[path[i], path[i + 1]]['weight']
    total_distance += graph.edges[path[-1], path[0]]['weight']
    return total_distance

In [4]:
def choose_reward( s, a):
  #reward를 반환하는 함수
    tot =0
    for j in range(num_states):
      if s!=j:
          tot+= G[s][j]['weight']
    tot = tot/(num_states-1)
    if G[s][a]['weight'] >tot:
        return -math.exp(G[s][a]['weight'])
    else:
        return -math.log(G[s][a]['weight'])

greedy tree search

In [20]:
def greedy_search(graph, start):
    n = len(graph.nodes)
    cur_visit_city = start
    visited = [start]
    final_cost = 0

    while len(visited) < n:
        min_weight = float('inf')
        next_city = None

        for neighbor in graph.neighbors(cur_visit_city):
            if neighbor not in visited:
                weight = graph.edges[cur_visit_city, neighbor]['weight']
                if weight < min_weight:
                    min_weight = weight
                    next_city = neighbor

        if next_city is None:
            break

        visited.append(next_city)
        final_cost += min_weight
        cur_visit_city = next_city

    return final_cost, visited

In [21]:
total_cost, path = greedy_search(G, 0)

In [22]:
total_cost += G[0][path[-1]]['weight']

A star tree search

In [5]:
from heapq import heappush, heappop
from copy import deepcopy
MAX_FRINGE_LIMIT = 8

#경로 구하기위한 astar와 heuristic
def heuristic_fuc(graph, sortedNodeList, city, visited):
  cur_city = city
  heu_cost = 0.0
  num = len(graph)

  while(1):
    #모든 노드 순회 확인
    if(len(visited)==num):
      return heu_cost + graph.edges[cur_city, 0]['weight']

    for weight, op_node in sortedNodeList[cur_city]:
      if op_node not in visited:
        heu_cost += weight
        visited.append(op_node)
        cur_city = op_node
        break

def a_star_search(g,sortedNodeList, start):
  #initialize
  global mx_cnt, path
  n = len(g)
  cur_visit_city = start
  visited = [start]
  pq = []
  #total_cost, total_cost - heu_cost, cur_city, visited_list
  heu = heuristic_fuc(g,sortedNodeList,cur_visit_city,deepcopy(visited))
  heappush(pq,(heu,0,cur_visit_city, deepcopy(visited)))
  final_cost = 0

  while(1):
    total_cost,cur_cost, cur_visit_city, visited = heappop(pq)

    #모든 노드 순회 했는지 확인
    if len(visited) == n:
        print(visited)
        path = visited
        final_cost = total_cost
        break

    cur_fringe_cnt = 0
    for weight, op_node in sortedNodeList[cur_visit_city]:

      if cur_fringe_cnt > MAX_FRINGE_LIMIT:
        break
      if op_node not in visited:
        visited.append(op_node)
        heu = heuristic_fuc(g,sortedNodeList, op_node, deepcopy(visited))
        heappush(pq,(heu + cur_cost + weight, cur_cost + weight, op_node, deepcopy(visited) ))
        cur_fringe_cnt+=1
        visited.remove(op_node)


  return final_cost



In [6]:
ans = a_star_search(G,sortedNodeList,0)

[0, 979, 373, 739, 21, 743, 776, 905, 865, 333, 970, 874, 962, 700, 814, 281, 157, 283, 401, 961, 399, 924, 731, 18, 976, 868, 985, 914, 363, 243, 772, 417, 684, 462, 492, 936, 439, 412, 851, 886, 805, 759, 501, 442, 930, 343, 713, 754, 177, 305, 402, 12, 72, 284, 465, 45, 974, 904, 244, 285, 310, 59, 112, 90, 336, 437, 414, 293, 698, 113, 91, 525, 602, 641, 560, 570, 556, 510, 514, 520, 648, 590, 506, 543, 575, 615, 673, 550, 569, 672, 631, 598, 555, 568, 620, 504, 591, 607, 536, 664, 503, 651, 566, 670, 509, 623, 632, 544, 655, 601, 531, 565, 645, 633, 577, 588, 527, 554, 538, 628, 562, 558, 535, 557, 518, 624, 537, 652, 578, 547, 619, 622, 541, 617, 604, 611, 637, 573, 666, 612, 563, 551, 549, 534, 630, 539, 515, 300, 356, 431, 485, 627, 320, 80, 331, 390, 136, 169, 455, 523, 532, 689, 639, 154, 493, 103, 348, 542, 371, 671, 545, 634, 519, 646, 644, 597, 528, 659, 610, 661, 662, 647, 660, 502, 546, 668, 614, 609, 508, 643, 596, 658, 533, 548, 579, 567, 559, 522, 638, 649, 640, 636, 

fringe 8일떄 path임

In [7]:
import numpy as np

# 상태 및 액션 수
num_states = len(path)
# 할인율
discount_factor = 0.9

# 초기 가치 함수 초기화
V = np.zeros(num_states)

for i in range(len(path)):
    state = path[i]
    total = 0
    c = 0

    # i부터 path의 끝까지 순회
    for j in range(i, len(path) - 1):
        total += G[path[j]][path[j+1]]['weight'] * (discount_factor ** c)
        c += 1

    # 마지막 노드에서 처음 노드로 가는 거리 추가
    total += G[path[-1]][path[0]]['weight'] * (discount_factor ** c)

    V[state] -= total

# 결과 출력
print("초기 가치 함수:")
for i in range(num_states):
    print("V({}) = {}".format(i, V[i]))

초기 가치 함수:
V(0) = -0.1770227427686099
V(1) = -0.2698182004649347
V(2) = -0.2027581808527208
V(3) = -0.22226202380776716
V(4) = -0.26961908951266755
V(5) = -0.18292537161073433
V(6) = -0.25157848835748814
V(7) = -0.6007467016198385
V(8) = -0.24745598039046926
V(9) = -0.16529438476149136
V(10) = -0.4937172969000997
V(11) = -0.1300332303288688
V(12) = -0.2243937923057544
V(13) = -0.19887398281000926
V(14) = -0.2440954561373679
V(15) = -0.19801766448990177
V(16) = -0.16751423855654807
V(17) = -0.1784957323656531
V(18) = -0.21025975661450927
V(19) = -0.1822956475653842
V(20) = -0.24875736347223487
V(21) = -0.15831090904474177
V(22) = -0.17882344159242
V(23) = -0.23592651400527226
V(24) = -0.3487827988597388
V(25) = -0.470941133893389
V(26) = -0.18875733921755436
V(27) = -0.17170506745435507
V(28) = -0.2062550486502045
V(29) = -0.228030859406162
V(30) = -0.2053652829758446
V(31) = -0.3691298695517203
V(32) = -0.2271048637367275
V(33) = -0.20005639014481644
V(34) = -0.2599878074830923
V(35) = 

In [8]:
import matplotlib.pyplot as plt
import numpy as np
iter_list = []
discount_factor = 0.9


# Value Iteration 함수 수정
def value_iteration(V, G, discount_factor, num_iterations, iter_list):
    for iteration in range(1, num_iterations + 1):

        for s in range(num_states):
            max_value = float("-inf")

            for next_s in range(num_states):
                if s == next_s:
                  continue
                value = -G[s][next_s]['weight'] + discount_factor * V[next_s]

                max_value = max(max_value, value)  # 최적 가치 갱신
            V[s] = max_value  # 새로운 가치 함수에 저장

        iter_list.append(V.copy())  # 가치 함수의 변화 기록 (복사하여 저장)

    return V, iter_list

# Value Iteration 수행
num_iterations = 50
V = np.zeros(num_states)  # 초기화
iter_list = []
V, iter_list = value_iteration(V, G, discount_factor, num_iterations, iter_list)

# 수렴 여부 확인
#이전 반복(iter_list[i-1])과 현재 반복(iter_list[i])의 가치 함수의 차이의 절대값 특정 임계값( 0.000001) 보다 작으면 수렴했구나 판단
converged = False
for i in range(1, len(iter_list)):
    if max(abs(iter_list[i] - iter_list[i-1])) < 1e-6:
        print(f"Converged at iteration {i}")
        converged = True
        break

if not converged:
    print("Did not converge within the specified number of iterations.")

Converged at iteration 49


In [9]:
# Q-table 초기화
import math
Q = np.zeros((num_states, num_states))
# 자기 자신 노드의 Q 값을 -무한대로 설정
for i in range(num_states):
    Q[i, i] = -np.inf

# Q-table 구성 => 거리가 먼 도시에 대해서는 패널티 많이주기
for s in range(num_states):

    weights = [(G[s][j]['weight'], j) for j in range(num_states) if s != j]
    weights.sort(reverse=True, key=lambda x: x[0])  # 가중치를 기준으로 내림차순 정렬
    tot = [j for _, j in weights[:40]]

    for a in range(num_states):

        if s != a:  # 자기 자신으로의 이동은 고려하지 않음
            if G[s][a]['weight'] in tot:
                Q[s, a] = -math.exp(G[s][a]['weight']) + discount_factor * V[a]
            else:
                Q[s, a] = -math.log(G[s][a]['weight']) + discount_factor * V[a]


In [10]:

paths = [0]  # 0번 노드부터 시작
current_node = 0

while len(paths) < num_states:
    max_value = float("-inf")
    max_node = -1

    for a in range(num_states):
        if a != current_node and a not in paths:
            if Q[current_node, a] > max_value:
                max_value = Q[current_node, a]
                max_node = a

    if max_node == -1:
        break  # 더 이상 선택할 노드가 없는 경우 루프 종료

    paths.append(max_node)
    current_node = max_node

print("paths:", paths)

paths: [0, 862, 66, 918, 944, 922, 971, 426, 764, 679, 736, 958, 965, 964, 748, 172, 937, 892, 933, 981, 386, 304, 680, 397, 229, 176, 11, 781, 961, 401, 399, 924, 731, 18, 976, 868, 985, 914, 363, 795, 706, 794, 685, 948, 968, 888, 750, 717, 699, 802, 900, 846, 866, 901, 788, 949, 721, 155, 137, 785, 796, 682, 104, 786, 214, 474, 464, 242, 910, 44, 950, 784, 283, 157, 281, 814, 700, 962, 874, 970, 739, 373, 776, 333, 865, 762, 905, 738, 712, 931, 727, 816, 987, 982, 991, 873, 729, 54, 878, 822, 941, 118, 237, 707, 875, 810, 249, 761, 254, 882, 342, 279, 978, 824, 915, 806, 315, 295, 9, 287, 799, 416, 955, 714, 765, 953, 787, 984, 834, 837, 830, 880, 694, 187, 989, 803, 919, 870, 461, 395, 876, 326, 942, 411, 98, 783, 895, 916, 760, 877, 726, 804, 701, 198, 227, 263, 959, 872, 232, 290, 735, 711, 40, 963, 768, 368, 206, 850, 751, 926, 913, 960, 747, 906, 730, 722, 725, 938, 840, 831, 678, 789, 732, 676, 952, 864, 883, 857, 853, 728, 813, 848, 675, 716, 26, 39, 920, 62, 308, 153, 841, 4

In [11]:
if len(paths) == len(set(paths)):
    print("All elements in paths are unique.")
else:
    print("There are duplicate elements in paths.")

All elements in paths are unique.


In [13]:
calculate_distance(G, paths)

25.986574325018

MC시작

A STAR 300개로 맞춤

In [14]:
from heapq import heappush, heappop
from copy import deepcopy
MAX_FRINGE_LIMIT = 8
mpath=[]
#경로 구하기위한 astar와 heuristic
def heuristic_fuc(graph, sortedNodeList, city, visited):
  cur_city = city
  heu_cost = 0.0
  num = len(graph)

  while(1):
    #모든 노드 순회 확인
    if(len(visited)==num):
      return heu_cost + graph.edges[cur_city, 0]['weight']

    for weight, op_node in sortedNodeList[cur_city]:
      if op_node not in visited:
        heu_cost += weight
        visited.append(op_node)
        cur_city = op_node
        break

def a_star_search_limit(g,sortedNodeList, start):
  #initialize
  global mx_cnt, mpath
#  n = 100
#n이 노드개수 뜻함 미리 만들 노드개수
  n = 700

  cur_visit_city = start
  visited = [start]
  pq = []
  #total_cost, total_cost - heu_cost, cur_city, visited_list
  heu = heuristic_fuc(g,sortedNodeList,cur_visit_city,deepcopy(visited))
  heappush(pq,(heu,0,cur_visit_city, deepcopy(visited)))
  final_cost = 0

  while(1):
    total_cost,cur_cost, cur_visit_city, visited = heappop(pq)

    #제한된 노드개수만큼 순회햇는지 확인
    if len(visited) == n:
        print(visited)
        mpath = visited
        final_cost = total_cost
        break

    cur_fringe_cnt = 0
    for weight, op_node in sortedNodeList[cur_visit_city]:

      if cur_fringe_cnt > MAX_FRINGE_LIMIT:
        break
      if op_node not in visited:
        visited.append(op_node)
        heu = heuristic_fuc(g,sortedNodeList, op_node, deepcopy(visited))
        heappush(pq,(heu + cur_cost + weight, cur_cost + weight, op_node, deepcopy(visited) ))
        cur_fringe_cnt+=1
        visited.remove(op_node)


  return final_cost

In [15]:
ansmc = a_star_search_limit(G,sortedNodeList,0)

[0, 979, 373, 739, 21, 743, 776, 905, 865, 333, 970, 874, 962, 700, 814, 281, 157, 283, 401, 961, 399, 924, 731, 18, 976, 868, 985, 914, 363, 243, 772, 417, 684, 462, 492, 936, 439, 412, 851, 886, 805, 759, 501, 442, 930, 343, 713, 754, 177, 305, 402, 12, 72, 284, 465, 45, 974, 904, 244, 285, 310, 59, 112, 90, 336, 437, 414, 293, 698, 113, 91, 525, 602, 641, 560, 570, 556, 510, 514, 520, 648, 590, 506, 543, 575, 615, 673, 550, 569, 672, 631, 598, 555, 568, 620, 504, 591, 607, 536, 664, 503, 651, 566, 670, 509, 623, 632, 544, 655, 601, 531, 565, 645, 633, 577, 588, 527, 554, 538, 628, 562, 558, 535, 557, 518, 624, 537, 652, 578, 547, 619, 622, 541, 617, 604, 611, 637, 573, 666, 612, 563, 551, 549, 534, 630, 539, 515, 300, 356, 431, 485, 627, 320, 80, 331, 390, 136, 169, 455, 523, 532, 689, 639, 154, 493, 103, 348, 542, 371, 671, 545, 634, 519, 646, 644, 597, 528, 659, 610, 661, 662, 647, 660, 502, 546, 668, 614, 609, 508, 643, 596, 658, 533, 548, 579, 567, 559, 522, 638, 649, 640, 636, 

In [16]:
print(len(mpath))

700


In [17]:
import numpy as np
import random
from collections import defaultdict

# 에이스타로 미리 몇개의 샘플 만들고 거기서부터 에피소드 만드는거
# 환경 변수 정의
num_states = 998  # 상태 수
num_actions = 998  # 행동 수
discount_factor = 0.9  # 할인 인자
learning_rate = 0.1  # 학습률
epsilon = 0.1  # 탐험(랜덤 행동) 확률



def generate_random_city(state, epath):
    nxt = np.random.randint(0, 998)
    while nxt == state or nxt in epath:
        nxt = np.random.randint(0, 998)
    return nxt

def generate_max_city(state, epath):
    #kpath = []
    jpath = [0] * 998
    #print(jpath)
    for i in epath:
    #  print(i)
      jpath[i] = 1
    #print(len(epath))
    possible_actions = np.argsort(Q[state])[::-1]  # Q 값이 높은 순으로 정렬된 인덱스 배열
    for action in possible_actions:
        if action != state and jpath[action]==1:  # 자기 자신이 아니고 epath에 없는 상태
            return action
    return generate_random_city(state, epath)  # 모든 행동이 이미 방문된 경우 무작위 선택

def choose_action(state, epath):
    if random.uniform(0, 1) < epsilon:
        return generate_random_city(state, epath)
    else:
        return generate_max_city(state, epath)

def generate_episode(cur_state, pi_state, epath):
    s_state = cur_state
    state = pi_state
    #print(s_state)
    #print(state)
    #episode = 0
    epipath = deepcopy(epath)
    gpath = [cur_state, pi_state]
    episode = []
    #total discount reward를 계산한다
    Gt= choose_reward(s_state, state)
    #c =0
    while True:
        next_state = choose_action(state, epipath)
        gpath.append(next_state)
        reward =  choose_reward(state, next_state)
        Gt+= reward
        #Gt += reward*(discount_factor**c)
        #episode += reward*(discount_factor**c)
        #episode += reward
        #c+=1
        state = next_state
        epipath.append(next_state)
        if len(epipath) > 997:  # 종료 조건 설정
            break
    #s,a, g  계산해서 넣어줌
    for i in range(len(gpath) - 1):
      s= gpath[i]
      a= gpath[i+1]
      episode.append((s, a, Gt))
      Gt-=choose_reward(s, a)

    return episode

def update_q_mc(epi_data):
    #st와 행동정책에 따른 다음 st+1이 동일한게 있는지 확인한다 =>평균내야하니까
    # 상태-행동 쌍과 해당 반환값을 저장할 딕셔너리 초기화
    state_action_returns = defaultdict(list)
    for state, action, reward in (epi_data):
      state_action_returns[(state, action)].append(reward)

    for key, values in state_action_returns.items():
        st, at = key  # 상태와 행동을 추출
        tot_gt = np.mean(values)  # 반환값들의 평균을 계산
        Q[st, at] += learning_rate * (tot_gt - Q[st, at])

# 에피소드 반복 (MC 기법을 활용)
num_episodes = 200

#트리서치 돌림 => 한 100, 200, 300 뭐 이정도까지 도시들
#나온경로들로부터 에피소드 업데이트
start_state = mpath[-1]
episode_data = []

for episode in range(num_episodes):
    #상태s를 경로에 없는걸로 랜덤하게 고른다
    cur_state = generate_random_city(start_state, mpath)

    epath = deepcopy(mpath)
    epath.append(cur_state)
    #print(len(epath))
    #행동정책 파이에 따라서 선택
    next_state = choose_action(cur_state, epath)
    #에피소드가 완료되어 받게되는 보상을 통해 q value update하는거
    # Monte Carlo 방식 에피소드 생성 및 Q-value 업데이트
    epath.append(next_state)
    #episode_data에는 gt값이 저장된다
    sm_episode = generate_episode(cur_state, next_state, epath)
    #print(sm_episode)
    print("episode:")
    print(episode + 1)
    #print(episode_data )
    episode_data.append(sm_episode)


    # 에피소드 종료 후 로그 출력
    if (episode + 1) % 100 == 0:
        print(f"Episode {episode + 1} completed.")

#에피소드들 끝나고 업데이트
print(episode_data)
#중첩리스트 풀기
flat_list = [item for sublist in episode_data for item in sublist]
print(flat_list)
#q value 업데이트
update_q_mc(flat_list)

# 최종 Q-테이블 출력
print("Final Q-Table:")
print(Q)


episode:
1
episode:
2
episode:
3
episode:
4
episode:
5
episode:
6
episode:
7
episode:
8
episode:
9
episode:
10
episode:
11
episode:
12
episode:
13
episode:
14
episode:
15
episode:
16
episode:
17
episode:
18
episode:
19
episode:
20
episode:
21
episode:
22
episode:
23
episode:
24
episode:
25
episode:
26
episode:
27
episode:
28
episode:
29
episode:
30
episode:
31
episode:
32
episode:
33
episode:
34
episode:
35
episode:
36
episode:
37
episode:
38
episode:
39
episode:
40
episode:
41
episode:
42
episode:
43
episode:
44
episode:
45
episode:
46
episode:
47
episode:
48
episode:
49
episode:
50
episode:
51
episode:
52
episode:
53
episode:
54
episode:
55
episode:
56
episode:
57
episode:
58
episode:
59
episode:
60
episode:
61
episode:
62
episode:
63
episode:
64
episode:
65
episode:
66
episode:
67
episode:
68
episode:
69
episode:
70
episode:
71
episode:
72
episode:
73
episode:
74
episode:
75
episode:
76
episode:
77
episode:
78
episode:
79
episode:
80
episode:
81
episode:
82
episode:
83
episode:
84
e

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
bpath = [0]  # 0번 노드부터 시작
current_node = 0

while len(bpath) < num_states:
    max_value = float("-inf")
    max_node = -1

    for a in range(num_states):
        if a != current_node and a not in bpath:
            if Q[current_node, a] > max_value:
                max_value = Q[current_node, a]
                max_node = a

    if max_node == -1:
        break  # 더 이상 선택할 노드가 없는 경우 루프 종료

    bpath.append(max_node)
    current_node = max_node

print("qpath:", bpath)

qpath: [0, 869, 222, 353, 1, 475, 884, 975, 728, 486, 50, 657, 505, 338, 144, 385, 166, 379, 194, 99, 756, 350, 224, 238, 117, 716, 891, 798, 396, 459, 524, 931, 608, 23, 753, 819, 422, 161, 589, 574, 280, 946, 818, 349, 375, 448, 301, 605, 77, 61, 758, 205, 572, 415, 13, 881, 60, 438, 245, 141, 193, 268, 318, 480, 428, 115, 220, 227, 991, 48, 29, 477, 148, 606, 478, 139, 366, 53, 35, 526, 81, 332, 434, 167, 307, 967, 718, 613, 156, 511, 433, 185, 278, 890, 979, 372, 170, 84, 256, 147, 239, 940, 263, 959, 473, 542, 150, 470, 107, 987, 697, 971, 172, 917, 337, 70, 844, 424, 997, 108, 460, 210, 423, 6, 149, 745, 361, 122, 321, 392, 204, 273, 10, 175, 654, 561, 25, 313, 7, 387, 939, 952, 964, 965, 254, 134, 322, 171, 183, 165, 587, 674, 311, 671, 250, 450, 762, 340, 618, 296, 282, 500, 695, 276, 463, 493, 103, 5, 76, 381, 33, 303, 482, 445, 932, 173, 146, 67, 369, 271, 270, 352, 629, 294, 595, 316, 37, 272, 425, 724, 274, 142, 14, 269, 184, 34, 848, 135, 121, 151, 858, 863, 440, 471, 21, 

In [19]:
calculate_distance(G, bpath) # 200


107.90489504795933